In [1]:
# huggingface
! pip install transformers

     |████████████████████████████████| 3.1 MB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 31.6 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 61 kB 468 kB/s 
     |████████████████████████████████| 596 kB 38.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# directory
!mkdir dataset

In [3]:
%cd /content/dataset

/content/dataset


In [4]:
# get data from github
!wget https://raw.githubusercontent.com/vohuy1894/NLP_Group/main/project/book%20text/215.txt

--2021-12-01 19:37:36--  https://raw.githubusercontent.com/vohuy1894/NLP_Group/main/project/book%20text/215.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 192108 (188K) [text/plain]
Saving to: ‘215.txt’

215.txt             100%[===================>] 187.61K  --.-KB/s    in 0.03s   

2021-12-01 19:37:37 (6.48 MB/s) - ‘215.txt’ saved [192108/192108]



In [5]:
# read the data 
# The Call of the Wild
text = open("215.txt", "r").read()

In [6]:
# preprocess data
text = text.replace('\\r'," ")
text = text.replace('\\n'," ")
text = text.replace('\\xe2\\x80\\x94',"-")
text = text.replace('\\xe2\\x80\\x9c','"')
text = text.replace('\\xe2\\x80\\x9d','"')
text = text.replace("\\xe2\\x80\\x99","'")
text[:1000]

'          Produced by Ryan, Kirstin, Linda and Rick Trapp, and David Widger          cover         The Call of the Wild        by Jack London        Contents       Chapter I. Into the Primitive   Chapter II. The Law of Club and Fang   Chapter III. The Dominant Primordial Beast   Chapter IV. Who Has Won to Mastership   Chapter V. The Toil of Trace   Chapter VI. For the Love of a Man   Chapter VII. The Sounding of the Call        Chapter I. Into the Primitive    "Old longings nomadic leap,  Chafing at custom\'s chain;  Again from its brumal sleep  Wakens the ferine strain."      Buck did not read the newspapers, or he would have known that trouble  was brewing, not alone for himself, but for every tide-water dog,  strong of muscle and with warm, long hair, from Puget Sound to San  Diego. Because men, groping in the Arctic darkness, had found a yellow  metal, and because steamship and transportation companies were booming  the find, thousands of men were rushing into the Northland. These

In [7]:
# split text into training and testing halves
test, train = text[:len(text)//2], text[len(text)//2:]

In [8]:
test[:1000]

'          Produced by Ryan, Kirstin, Linda and Rick Trapp, and David Widger          cover         The Call of the Wild        by Jack London        Contents       Chapter I. Into the Primitive   Chapter II. The Law of Club and Fang   Chapter III. The Dominant Primordial Beast   Chapter IV. Who Has Won to Mastership   Chapter V. The Toil of Trace   Chapter VI. For the Love of a Man   Chapter VII. The Sounding of the Call        Chapter I. Into the Primitive    "Old longings nomadic leap,  Chafing at custom\'s chain;  Again from its brumal sleep  Wakens the ferine strain."      Buck did not read the newspapers, or he would have known that trouble  was brewing, not alone for himself, but for every tide-water dog,  strong of muscle and with warm, long hair, from Puget Sound to San  Diego. Because men, groping in the Arctic darkness, had found a yellow  metal, and because steamship and transportation companies were booming  the find, thousands of men were rushing into the Northland. These

In [9]:
train[:1000]

'st dollars, they were  to be sold.    Three days passed, by which time Buck and his mates found how really  tired and weak they were. Then, on the morning of the fourth day, two  men from the States came along and bought them, harness and all, for a  song. The men addressed each other as "Hal" and "Charles." Charles was  a middle-aged, lightish-colored man, with weak and watery eyes and a  mustache that twisted fiercely and vigorously up, giving the lie to the  limply drooping lip it concealed. Hal was a youngster of nineteen or  twenty, with a big Colt\'s revolver and a hunting-knife strapped about  him on a belt that fairly bristled with cartridges. This belt was the  most salient thing about him. It advertised his callowness-a callowness  sheer and unutterable. Both men were manifestly out of place, and why  such as they should adventure the North is part of the mystery of  things that passes understanding.    Buck heard the chaffering, saw the money pass between the man and the  G

In [10]:
# save output and reupload as txt files

In [11]:
# generate tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

train_path = '/content/dataset/train.txt'
test_path = '/content/dataset/test.txt'

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [12]:
# data collator
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train,test,tokenizer):
  train_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=train_path,
      block_size=128)
  
  test_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=test_path,
      block_size=128)
  
  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer, mlm=False,
  )
  return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (21516 > 1024). Running this sequence through the model will result in indexing errors


In [13]:
# trainer from huggingface
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2-medium")

training_args = TrainingArguments(
    output_dir="./gpt2-medium_Call_of_the_Wild_fine_tuned",  #output directory
    overwrite_output_dir=True,
    num_train_epochs=2, # training epochs
    per_device_train_batch_size=8, # batch size for training 
    per_device_eval_batch_size=64, # batch size for evaluation
    eval_steps = 400, # number of update steps between 2 evals
    save_steps = 800, # when model is saved
    warmup_steps = 500, # warump steps for learning rate scheduler
    prediction_loss_only = True

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:698: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [14]:
# cuda running out memory; adjust hyperparameters?
trainer.train()

***** Running training *****
  Num examples = 168
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 42


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=42, training_loss=4.19997805640811, metrics={'train_runtime': 71.0337, 'train_samples_per_second': 4.73, 'train_steps_per_second': 0.591, 'total_flos': 78010858340352.0, 'train_loss': 4.19997805640811, 'epoch': 2.0})

In [15]:
trainer.save_model()

Saving model checkpoint to ./gpt2-medium_Call_of_the_Wild_fine_tuned
Configuration saved in ./gpt2-medium_Call_of_the_Wild_fine_tuned/config.json
Model weights saved in ./gpt2-medium_Call_of_the_Wild_fine_tuned/pytorch_model.bin


In [16]:
# text generation with fine tuned model
from transformers import pipeline

text = "By the time he came home, it was still snowing."
model = pipeline('text-generation', model='./gpt2-medium_Call_of_the_Wild_fine_tuned', tokenizer='gpt2-medium')
output = model(text)[0]['generated_text']

loading configuration file ./gpt2-medium_Call_of_the_Wild_fine_tuned/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transf

In [17]:
# text output
output

"By the time he came home, it was still snowing. A thin trickle of ice pooled around his ankles and dried onto his skin until it broke away. The snow-covered porch was soaked up by the heat. That morning, he'd been"